# Word Embeddings

## Representación de palabras mediante vectores

## Créditos

Este material está fuertemente inspirado en los cursos de Stanford [CS224n: Natural Language Processing with Deep Learning](http://web.stanford.edu/class/cs224n/) y [CS224u: Natural Language Understanding](http://web.stanford.edu/class/cs224u/), así también como del [blog de Jay Alammar](http://jalammar.github.io/).

## Conceptos básicos de representación de palabras

### Características de las palabras

- Suelen ser **unidad básica** en cualquier tarea de procesamiento de lenguaje natural (PLN).
    - Hoy en día, cuando hablamos de "palabras" podemos referirnos a otros símbolos como subpalabras o incluso conjuntos de caracteres.
- Gran parte del trabajo de PLN solía representar las palabras como **símbolos atómicos**.
    - Esto ya no es así salvo algunas excepciones.
- Cualquier modelo de PLN requiere como entrada la **representación de una palabra**.
- Las nociones de **similitud y distancia** entre palabras son cruciales para tareas de PLN.

### Representación discreta

- Para aplicar medidas de similitud y distancia (e.g. euclídea, coseno), utilizamos **vectores que representen palabras**.
- Primera aproximación: **vectores *one-hot***.
- Para cada palabra del vocabulario, tenemos un vector.
- En términos de espacio vectorial, el vector tiene un **1** en una de sus dimensiones y __0s__ en todas las demás.
- **Dimensionalidad muy alta** (el inglés tiene un estimado de 13 millones de palabras).
- Los vectores son **ralos** (o esparsos), tienen muchos ceros.

### Ejemplo de representación discreta

Dado un corpus:

- "El fallo fue decisivo"
- "La corte rectificará el fallo"

Tenemos un vocabulario $V = \{corte, decisivo, el, fallo, fue, la, rectificará\}$.

Codificamos un número $|V| = 7$ de vectores para cada palabra $w^{(i)} \in \mathbb{R}^{|V|}$.

### Ejemplo de representación discreta

$$
    w^{corte} =
    \begin{bmatrix}
       1 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0
    \end{bmatrix} ,
    w^{decisivo} =
    \begin{bmatrix}
       0 \\ 1 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0
    \end{bmatrix} ,
    w^{el} =
    \begin{bmatrix}
       0 \\ 0 \\ 1 \\ 0 \\ 0 \\ 0 \\ 0
    \end{bmatrix} ,
    w^{fallo} =
    \begin{bmatrix}
       0 \\ 0 \\ 0 \\ 1 \\ 0 \\ 0 \\ 0
    \end{bmatrix} ,
$$

$$
    w^{fue} =
    \begin{bmatrix}
       0 \\ 0 \\ 0 \\ 0 \\ 1 \\ 0 \\ 0
    \end{bmatrix}
    w^{la} =
    \begin{bmatrix}
       0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 1 \\ 0
    \end{bmatrix} ,
    w^{rectificará} =
    \begin{bmatrix}
       0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 1
    \end{bmatrix}
$$

In [ ]:
import numpy as np

corpus = [
    "El fallo fue decisivo", 
    "La corte rectificará el fallo"
]

tokenized_corpus = [
    [word.lower() for word in doc.split()] for doc in corpus
]

vocabulary = sorted(set([
    word for doc in tokenized_corpus for word in doc
]))

one_hot_vectors = np.eye(len(vocabulary), dtype=np.int32)

for doc in tokenized_corpus:
    print(np.array([
        one_hot_vectors[vocabulary.index(word), :] for word in doc
    ]))

## Representación distribuída de palabras

### Concepto

- Idea: Representar una palabra **mediante sus vecinos**.
    - Es una de las **ideas más importantes** en PLN moderno.

#### Ejemplo: 

- "_El gobierno rescató a los_ **bancos** _ante la crisis de crédito_".
- "_La municipalidad reparará los_ **bancos** _de la plaza_".

En estos ejemplos, las palabras que están en _cursiva_ representarán las palabras en **negrita**.

### Motivación

- La representación distribuída nos da pie a pensar como los vectores **codifican el significado de las unidades linguísticas** (palabras, oraciones, documentos, etc.).
- Son la base de los **modelos de espacio vectorial**.
- Piezas fundamentales en modelos para **Procesamiento y Comprensión de Lenguaje Natural**.
- Estas representaciones pueden ser usadas, entre otros, para:
    - Entender y modelar fenómenos sociales y lingüísticos.
    - Entrada para modelos de aprendizaje automático.

## Representación mediante matrices

### Objetivos del diseño de matrices

- Buscamos representar palabras de manera matemática.
    - Idea: Mediante una matriz (vista como un conjunto de vectores)
- Hay que definir la construcción de una matriz.
    - Muchas opciones de diseño.
    - Distinto impacto tendrán las decisiones tomadas respecto al modelado del texto.

### Matriz de co-ocurrencia de palabras

- Cada palabra se representa por las palabras a su alrededor (dada una ventana).
- La matriz es simétrica (independiente de la posición de las palabras).
    - Esto se conoce como "bolsa de palabras" (o _bag of words_)
- Dado un vocabulario $V$:
    - Matriz de co-ocurrencias $X \in \mathbb{R}^{|V|\times|V|}$
    - $X_{ij}$ representa la co-ocurrencia entre la palabras $w^{(i)}$ y $w^{(j)}$

### Matriz de co-ocurrencia de palabras

<img src="images/word-word-matrix.png" width="90%"/>
<div style="text-align: right; font-size:.9em;">Source: <a href="http://web.stanford.edu/class/cs224u/" target="_blank">http://web.stanford.edu/class/cs224u/</a></div>

### Matriz de documentos

- Cada documento se representa por las palabras que lo conforman.
- Se usa un modelo de bolsa de palabras: no importa la posición.
- Dadas $N$ palabras en un corpus de $M$ documentos:
    - Matriz de documentos $X \in \mathbb{R}^{N \times M}$
    - $X_{ij}$ representa la ocurrencia de la palabra $w^{(i)}$ en el documento $d^{(j)}$.

### Matriz de documentos

<img src="images/term-document-matrix.png" width="90%"/>
<div style="text-align: right; font-size:.9em;">Source: <a href="http://web.stanford.edu/class/cs224u/" target="_blank">http://web.stanford.edu/class/cs224u/</a></div>

### Ventana y ponderación

- La ventana define la **cantidad de palabras en el contexto** a considerar.
- La ponderación (*scaling*) le da un **peso a la palabra** en la suma total.
    - Igual peso para todas las palabras, i.e. bolsa de palabras.
    - Peso distinto de acuerdo a la distancia, e.g. decaimiento fraccional.
    - Peso binario, i.e. la palabra está o no.

<img src="images/window-scaling.png" width="90%">
<div style="text-align: right; font-size:.9em;">Source: <a href="http://web.stanford.edu/class/cs224u/" target="_blank">http://web.stanford.edu/class/cs224u/</a></div>

### Ventana y ponderación

- Ventanas más grandes y con peso igual (_flat_) capturan **información semántica**.
    - Las matrices de documentos dan idea de la temática del documento.
- Ventanas más cortas y con pesos capturan **información sintáctica** (colocacional).
    - Las matrices de co-ocurrencias de palabras son de afinidad de palabras.
- El límite puede definirse más allá de una ventana fija.
    - Utilizar oraciones, párrafos o documentos tendrá consecuencias distintas.

### Consideraciones

- Las matrices **incrementan con el vocabulario**.
    - Definir límites en vocabulario o bien lidiar con modelos muy grandes.
    - Por la ley de Zipf (Zipf, 1949), la lista de palabras con escasa aparición se vuelve muy grande.
- La elección de la ventana afecta en la obtención de **matrices más o menos dispersas**.
- Representar distintas unidades lingüísticas deriva en **más dispersión** (ver el ejemplo de word-word vs term-document).

## Reducción de dimensionalidad

### Objetivos de reducir dimensiones

- Buscamos asociaciones de alto orden entre los datos.
    - Estas no siempre están presentes en los conteos o los esquemas de reponderación.
- Se busca capturar nociones de co-ocurrencia de mayor orden.
- Podemos considerar dos palabras: "bizcocho" y "criollito". 
    - En muchos esquemas pueden referirse a lo mismo.
    - "Bizcocho" es más común en Buenos Aires, mientras que "criollito" es de Córdoba.
    - Pueden que en un mismo texto no aparezca nunca juntas.
    - Queremos capturar la correlación entre ambas por más que la co-ocurrencia sea 0.
- Las técnicas de reducción de dimensionalidad suelen lograr encontrar similaridades semánticas en segundo plano.
- Por otro lado reducen el tamaño de los modelos y los hacen menos dispersos.

### Análisis de Semántica Latente (LSA)

- Es uno de los modelos de reducción de dimensionalidad que más se ha utilizado (Deerwester et al., 1990).
- Consiste en aplicar **descomposición de valores singulares truncada** (_truncated SVD_), de álgebra lineal, sobre los VSMs.
- Establece un baseline bastante estándar y muchas veces difícil de superar.
- Con un vocabulario muy grande tiene problemas para escalar pues tiene costo cuadrático.

### Algoritmo de LSA

- Generar la matriz del VSM $X$
- Aplicar SVD sobre $X$ de manera que $X = USW^{T}$.
- $U$ y $W$ son matrices ortonormales: Sus columnas están normalizadas por longitud y son ortogonales entre si (la distancia coseno es 1).
- $S$ es una matriz diagonal con valores singulares ordenados por tamaño (la primera dimensión corresponde a la fuente de mayor variabilidad en los datos, y así sucesivamente).
- normalizadas en longitud, y ortogonales la una de la otra
- Observando los _valores singulares_ los truncamos en un índice $k$.
- Tomamos $\hat{X} = U_{1:|V|,1:k}S_{1:k,1:k}$, donde $|V|$ es el tamaño de nuestro vocabulario.

### Algoritmo de LSA

<img src="images/lsa.png" width="90%" />

### Problemas de LSA

- Las palabras "vacías" (artículos, pronombres, conectores, etc.) son muy frecuentes.
    - Se puede ignorar o limitar.
    - Se puede reponderar las palabras mediante técnicas como [PMI](https://en.wikipedia.org/wiki/Pointwise_mutual_information) o [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf).
- La dimensión de la matriz puede cambiar a medida que hay nuevas palabras, cambiando el temaño del vocuabulario $|V|$.
- Las matrices de co-ocurrencia suelen ser muy ralas.
- El costo de SVD es cuadrático.

## Word Embeddings

### Vectores densos de palabras

- La idea es directamente **aprender vectores densos**.
- Se crea un modelo que puede ser **entrenado de manera iterativa**.
- El modelo codifica la **probabilidad de una palabra dado su contexto**.

### Predecir la siguiente palabra

<img src="images/next-word.png" width="90%">
<div style="text-align: right; font-size:.9em;">Source: <a href="https://jalammar.github.io/illustrated-word2vec/" target="_blank">https://jalammar.github.io/illustrated-word2vec/</a></div>

### Modelos de lenguaje

- Dada una secuencia de palabras de entrada, el modelo se entrena para predecir la palabra que continúa la oración.

<img src="images/language-model.png" width="90%">
<div style="text-align: right; font-size:.9em;">Source: <a href="https://jalammar.github.io/illustrated-word2vec/" target="_blank">https://jalammar.github.io/illustrated-word2vec/</a></div>

### Embedding Lookup

- Los embeddings se generan en una **lookup matrix**.
- Las dimensiones de la matriz representan **rasgos latentes** de las palabras.
- Los rasgos son pesos, se optimizan para mejorar la predicción del modelo.

<img src="images/language-model-embedding.png" width="90%">
<div style="text-align: right; font-size:.9em;">Source: <a href="https://jalammar.github.io/illustrated-word2vec/" target="_blank">https://jalammar.github.io/illustrated-word2vec/</a></div>

### Generación de datos: Autosupervisados

- El **texto libre** se puede utilizar para generar datos.
- Son datos no supervisados transformados a una tarea supervisada: Esto se conoce como **autosupervisión o self-supervision**.
- A partir de una **ventana** generamos nuestros datos de entrada y la etiqueta (palabra) de salida.

<img src="images/lm-dataset.png" width="90%">
<div style="text-align: right; font-size:.9em;">Source: <a href="https://jalammar.github.io/illustrated-word2vec/" target="_blank">https://jalammar.github.io/illustrated-word2vec/</a></div>

# Word2vec

### Word2vec: Un algoritmo para calcular embeddings

- Surge del **trabajo de Mikolov (2013)**.
- La idea principal es **predecir una palabra dado su contexto**.
- Se presentaron dos maneras de realizarlo:
    - CBOW (Continous Bag-of-Words): Dado un contexto **prededir la palabra central**.
    - Skipgram: Dada la palabra centrar **predecir el contexto**.

### Continous Bag-of-Words

<img src="images/cbow-1.png" width="90%">
<img src="images/cbow-2.png" width="90%">
<div style="text-align: right; font-size:.9em;">Source: <a href="https://jalammar.github.io/illustrated-word2vec/" target="_blank">https://jalammar.github.io/illustrated-word2vec/</a></div>

### Skipgram

<img src="images/skipgram.png" width="90%">
<div style="text-align: right; font-size:.9em;">Source: <a href="https://jalammar.github.io/illustrated-word2vec/" target="_blank">https://jalammar.github.io/illustrated-word2vec/</a></div>

### Entrenamiento

- Generamos el conjunto de datos.
- El modelo arranca con pesos aleatorios.
- Se entrenará con sucesivas iteraciones.

<img src="images/word2vec-1.png" width="90%">
<div style="text-align: right; font-size:.9em;">Source: <a href="https://jalammar.github.io/illustrated-word2vec/" target="_blank">https://jalammar.github.io/illustrated-word2vec/</a></div>

### Entrenamiento

<img src="images/word2vec-2.png" width="90%">
<div style="text-align: right; font-size:.9em;">Source: <a href="https://jalammar.github.io/illustrated-word2vec/" target="_blank">https://jalammar.github.io/illustrated-word2vec/</a></div>

### Negative Sampling

- El hecho de tener que proyectar a todo el vocuabulario hace que el entrenamiento se vuelva caro.

<img src="images/neg-sam-1.png" width="90%">
<div style="text-align: right; font-size:.9em;">Source: <a href="https://jalammar.github.io/illustrated-word2vec/" target="_blank">https://jalammar.github.io/illustrated-word2vec/</a></div>

### Negative Sampling: Cambio de tarea

<img src="images/neg-sam-2.png" width="90%">
<div style="text-align: right; font-size:.9em;">Source: <a href="https://jalammar.github.io/illustrated-word2vec/" target="_blank">https://jalammar.github.io/illustrated-word2vec/</a></div>

### Negative Sampling: Nueva tarea

<img src="images/neg-sam-3.png" width="90%">
<div style="text-align: right; font-size:.9em;">Source: <a href="https://jalammar.github.io/illustrated-word2vec/" target="_blank">https://jalammar.github.io/illustrated-word2vec/</a></div>

### Negative Sampling: Cambio de labels

<img src="images/neg-sam-4.png" width="90%">
<div style="text-align: right; font-size:.9em;">Source: <a href="https://jalammar.github.io/illustrated-word2vec/" target="_blank">https://jalammar.github.io/illustrated-word2vec/</a></div>

### Negative Sampling: Agregar ejemplos negativos

<img src="images/neg-sam-5.png" width="90%">
<div style="text-align: right; font-size:.9em;">Source: <a href="https://jalammar.github.io/illustrated-word2vec/" target="_blank">https://jalammar.github.io/illustrated-word2vec/</a></div>

### Word2vec: Matrices

<img src="images/word2vec-3.png" width="90%">
<div style="text-align: right; font-size:.9em;">Source: <a href="https://jalammar.github.io/illustrated-word2vec/" target="_blank">https://jalammar.github.io/illustrated-word2vec/</a></div>

### Word2vec: Viendo un batch

<img src="images/word2vec-4.png" width="90%">
<div style="text-align: right; font-size:.9em;">Source: <a href="https://jalammar.github.io/illustrated-word2vec/" target="_blank">https://jalammar.github.io/illustrated-word2vec/</a></div>

### Word2vec: Obteniendo los embeddings

<img src="images/word2vec-5.png" width="90%">
<div style="text-align: right; font-size:.9em;">Source: <a href="https://jalammar.github.io/illustrated-word2vec/" target="_blank">https://jalammar.github.io/illustrated-word2vec/</a></div>

### Word2vec: Estimando el error

<img src="images/word2vec-6.png" width="90%">
<div style="text-align: right; font-size:.9em;">Source: <a href="https://jalammar.github.io/illustrated-word2vec/" target="_blank">https://jalammar.github.io/illustrated-word2vec/</a></div>

### Word2vec: Vectores finales

- Finalizamos con dos vectores por cada palabra: los de input y los de output.
    - Estos representan la palabra central y/o el contexto de acuerdo al modelo (CBOW vs. Skipgram).
- Ambos vectores tienen información de coocurrencia.
- ¿Qué hacemos para obtener un vector final?

### Word2vec: Algunas propiedades

- Las dimensiones de los embeddings son latentes. 
    - Codifican información, pero son difíciles de interpretar.
- Los vectores suelend codificar dimensiones de similitud.
- Aplicando operaciones aritméticas obtenemos valores interesantes en los resultados.
- Son entrenados de manera supervisada y pueden servir como input a otros modelos de aprendizaje supervisado.

<img src="images/king-analogy.png" width="90%">
<div style="text-align: right; font-size:.9em;">Source: <a href="https://jalammar.github.io/illustrated-word2vec/" target="_blank">https://jalammar.github.io/illustrated-word2vec/</a></div>

### Word2Vec: Ejemplo via Gensim

In [ ]:
!gunzip < ./data/glove.6B.50d.txt.gz > ./data/glove.6B.50d.txt
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format("./data/glove.6B.50d.txt", binary=False, no_header=True)

In [ ]:
print(model.most_similar(model["king"], topn=3))

In [ ]:
print(model.most_similar(model["king"] - model["man"] + model["woman"], topn=3))

In [ ]:
print(model.most_similar(model["boy"] - model["man"] + model["woman"], topn=3))

## Referencias

- Deerwester, S., Dumais, S. T., Furnas, G. W., Landauer, T. K., & Harshman, R. (1990). Indexing by latent semantic analysis. Journal of the American society for information science, 41(6), 391-407.
- Mikolov, T., Chen, K., Corrado, G., & Dean, J. (2013). Efficient estimation of word representations in vector space. arXiv preprint arXiv:1301.3781.
- Zipf, G. K. (1949). Human behavior and the principle of least effort.